In [2]:
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
import numpy as np

In [3]:

df = pd.read_csv('https://gist.githubusercontent.com/sachinsdate/09cfd42b7701c48ec68b04c786786434/raw/4b50e718a83a4b20adcff5552ccea8f1054ce919/fish.csv')

In [4]:
df.head()

,LIVE_BAIT,CAMPER,PERSONS,CHILDREN,FISH_COUNT
0,0,0,1,0,0
1,1,1,1,0,0
2,1,0,1,0,0
3,1,1,2,1,0
4,1,0,1,0,1


In [5]:
px.bar(df['FISH_COUNT'].value_counts())

In [6]:
df.query('FISH_COUNT == 0').shape[0]/df.shape[0]

0.568

In [7]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=df.index, y=df.FISH_COUNT),row=1, col=1)
fig.add_trace(go.Scatter(x=df.query('FISH_COUNT != 0').index, y=df.query('FISH_COUNT != 0').FISH_COUNT),row=1, col=2)


In [8]:
#define o ponto de corte do dataset de treino

def create_train_test(df):
    
    cut = 0.8
    index_cut = int(len(df) * cut)
    df_train= df.iloc[:index_cut,:]
    df_test = df.iloc[index_cut:,:]
    return (df_train, df_test)


In [9]:
df_train, df_test  = create_train_test(df)

In [10]:
from patsy import dmatrices

expr = 'FISH_COUNT ~ LIVE_BAIT  + CAMPER + CHILDREN + PERSONS'

y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')

y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')



In [11]:

#zero inflated poisson
#y_train = df_train['FISH_COUNT']
#X_train = df_train.drop('FISH_COUNT', axis=1)
zip_training_results = sm.ZeroInflatedPoisson(endog=y_train, 
                                              exog=X_train, 
                                              exog_infl=X_train.drop(['Intercept', 'LIVE_BAIT','CAMPER'], axis=1),
                                                inflation='logit').fit()



Optimization terminated successfully.
         Current function value: 3.214268
         Iterations: 21
         Function evaluations: 23
         Gradient evaluations: 23


In [12]:
print(zip_training_results.summary())

                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:              FISH_COUNT   No. Observations:                  200
Model:             ZeroInflatedPoisson   Df Residuals:                      195
Method:                            MLE   Df Model:                            4
Date:                 Thu, 08 Jun 2023   Pseudo R-squ.:                  0.3449
Time:                         16:18:13   Log-Likelihood:                -642.85
converged:                        True   LL-Null:                       -981.37
Covariance Type:             nonrobust   LLR p-value:                3.258e-145
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
inflate_CHILDREN     1.9392      0.364      5.328      0.000       1.226       2.653
inflate_PERSONS     -0.5669      0.113     -5.035      0.000      -0.788      -0.346
Intercept           

In [13]:
zip_predictions = zip_training_results.predict(X_test,exog_infl=X_test.drop(['Intercept', 'LIVE_BAIT','CAMPER'], axis=1))

In [14]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_test.index, y=df_test.FISH_COUNT, name='obs'))
fig.add_trace(go.Scatter(x=zip_predictions.index, y=zip_predictions, name='pred'))

In [25]:
import sklearn

ModuleNotFoundError: No module named 'sklearn'